# ARIMA Model for Stock Return Prediction

This notebook implements an ARIMA model for predicting stock returns using the hackathon dataset.


In [1]:
import datetime
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path for data_loader import
sys.path.append(str(Path.cwd().parent / 'src'))
from data_loader import load_data


/var/folders/bn/cx_ppcpx0ngbpf4h354g9_k80000gn/T/ipykernel_38877/1606108990.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# For timing purpose
print(f"Start time: {datetime.datetime.now()}")

# Turn off pandas Setting with Copy Warning
pd.set_option("mode.chained_assignment", None)


Start time: 2025-09-21 22:09:58.836471


In [ ]:
# Load data using data_loader
print("Loading data...")
raw = load_data(filename="ret_sample.csv", parse_dates=["ret_eom"], low_memory=False)
print(f"Data shape: {raw.shape}")
print(f"Date range: {raw['ret_eom'].min()} to {raw['ret_eom'].max()}")

# Ensure the date column is properly converted to datetime
raw['date'] = pd.to_datetime(raw['ret_eom'])
print(f"Date column type: {raw['date'].dtype}")

# Load list of predictors for stocks
stock_vars = list(load_data(filename="factor_char_list.csv")["variable"].values)
print(f"Number of predictor variables: {len(stock_vars)}")

Loading data...
Loading data from: /Users/kevin/Coding Projects/Asset-Management-Hackathon-2025/data/ret_sample.csv


In [ ]:
# Define the left hand side variable
ret_var = "stock_ret"
new_set = raw[raw[ret_var].notna()].copy()
print(f"Data after removing missing returns: {new_set.shape}")

# Transform each variable in each month to the same scale
monthly = new_set.groupby("date")
data = pd.DataFrame()

for date, monthly_raw in monthly:
    group = monthly_raw.copy()
    # Rank transform each variable to [-1, 1]
    for var in stock_vars:
        var_median = group[var].median(skipna=True)
        group[var] = group[var].fillna(var_median)  # Fill missing values with cross-sectional median
        
        group[var] = group[var].rank(method="dense") - 1
        group_max = group[var].max()
        if group_max > 0:
            group[var] = (group[var] / group_max) * 2 - 1
        else:
            group[var] = 0  # In case of all missing values
            print(f"Warning: {date}, {var} set to zero.")
    
    # Add the adjusted values
    data = pd.concat([data, group], ignore_index=True)

print(f"Processed data shape: {data.shape}")
print(f"Date column in processed data type: {data['date'].dtype}")

In [ ]:
# Function to check stationarity
def check_stationarity(series, title=''):
    result = adfuller(series.dropna())
    print(f'{title} ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    return result[1] < 0.05  # Return True if stationary

# Function to find optimal ARIMA parameters
def find_arima_params(series, max_p=3, max_d=2, max_q=3):
    best_aic = np.inf
    best_params = None
    
    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    model = ARIMA(series, order=(p, d, q))
                    fitted_model = model.fit()
                    if fitted_model.aic < best_aic:
                        best_aic = fitted_model.aic
                        best_params = (p, d, q)
                except:
                    continue
    
    return best_params, best_aic


In [ ]:
# Initialize the starting date, counter, and output data
starting = pd.to_datetime("20050101", format="%Y%m%d")
counter = 0
pred_out = pd.DataFrame()

print("Starting ARIMA model training and prediction...")

# Estimation with expanding window
while (starting + pd.DateOffset(years=11 + counter)) <= pd.to_datetime("20260101", format="%Y%m%d"):
    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),  # Use 8 years and expanding as training set
        starting + pd.DateOffset(years=10 + counter),  # Use next 2 years as validation set
        starting + pd.DateOffset(years=11 + counter),
    ]  # Use next year as out-of-sample testing set
    
    print(f"\nProcessing period {counter + 1}: {cutoff[0].strftime('%Y-%m-%d')} to {cutoff[3].strftime('%Y-%m-%d')}")
    
    # Cut the sample into training, validation, and testing sets
    train = data[(data["date"] >= cutoff[0]) & (data["date"] < cutoff[1])]
    validate = data[(data["date"] >= cutoff[1]) & (data["date"] < cutoff[2])]
    test = data[(data["date"] >= cutoff[2]) & (data["date"] < cutoff[3])]
    
    print(f"Training set: {len(train)} observations")
    print(f"Validation set: {len(validate)} observations")
    print(f"Test set: {len(test)} observations")
    
    # Prepare output data
    reg_pred = test[["year", "month", "ret_eom", "id", ret_var]].copy()
    
    # For ARIMA, we'll work with the time series of returns
    # We'll create a time series for each stock and predict individually
    arima_predictions = []
    
    # Get unique stocks in test set
    test_stocks = test['id'].unique()
    print(f"Predicting for {len(test_stocks)} stocks...")
    
    for stock_id in test_stocks:
        # Get historical data for this stock
        stock_train = train[train['id'] == stock_id].sort_values('date')
        stock_val = validate[validate['id'] == stock_id].sort_values('date')
        stock_test = test[test['id'] == stock_id].sort_values('date')
        
        if len(stock_train) < 10:  # Need minimum observations for ARIMA
            # Use simple mean prediction if not enough data
            mean_ret = stock_train[ret_var].mean() if len(stock_train) > 0 else 0
            arima_pred = [mean_ret] * len(stock_test)
        else:
            # Prepare time series
            train_series = stock_train[ret_var].values
            
            # Check stationarity
            is_stationary = check_stationarity(pd.Series(train_series), f'Stock {stock_id}')
            
            # Find optimal ARIMA parameters
            best_params, best_aic = find_arima_params(pd.Series(train_series))
            
            if best_params is not None:
                try:
                    # Fit ARIMA model
                    model = ARIMA(train_series, order=best_params)
                    fitted_model = model.fit()
                    
                    # Make predictions
                    arima_pred = fitted_model.forecast(steps=len(stock_test))
                    
                    # If forecast returns a single value, repeat it
                    if len(arima_pred) == 1:
                        arima_pred = [arima_pred[0]] * len(stock_test)
                    
                except Exception as e:
                    print(f"Error fitting ARIMA for stock {stock_id}: {e}")
                    # Fallback to mean prediction
                    mean_ret = stock_train[ret_var].mean()
                    arima_pred = [mean_ret] * len(stock_test)
            else:
                # Fallback to mean prediction
                mean_ret = stock_train[ret_var].mean()
                arima_pred = [mean_ret] * len(stock_test)
        
        # Store predictions
        for i, (_, row) in enumerate(stock_test.iterrows()):
            if i < len(arima_pred):
                arima_predictions.append(arima_pred[i])
            else:
                arima_predictions.append(arima_pred[-1] if arima_pred else 0)
    
    # Add ARIMA predictions to output
    reg_pred["arima"] = arima_predictions
    
    # Add to the output data
    pred_out = pd.concat([pred_out, reg_pred], ignore_index=True)
    
    # Go to the next year
    counter += 1
    
    # Break if we've processed enough periods (optional limit)
    if counter >= 5:  # Limit for demonstration
        break

print(f"\nCompleted processing {counter} periods")


In [ ]:
# Output the predicted values to CSV
output_path = Path.cwd().parent / "data" / "arima_predictions.csv"
print(f"Saving predictions to: {output_path}")
pred_out.to_csv(output_path, index=False)

print(f"\nPrediction results shape: {pred_out.shape}")
print(f"Columns: {list(pred_out.columns)}")


In [ ]:
# Calculate and print the OOS R²
yreal = pred_out[ret_var].values
ypred_arima = pred_out["arima"].values

# Calculate R²
r2_arima = 1 - np.sum(np.square((yreal - ypred_arima))) / np.sum(np.square(yreal))
print(f"ARIMA OOS R²: {r2_arima:.6f}")

# Calculate additional metrics
mse_arima = mean_squared_error(yreal, ypred_arima)
rmse_arima = np.sqrt(mse_arima)
mae_arima = np.mean(np.abs(yreal - ypred_arima))

print(f"ARIMA MSE: {mse_arima:.6f}")
print(f"ARIMA RMSE: {rmse_arima:.6f}")
print(f"ARIMA MAE: {mae_arima:.6f}")


In [ ]:
# Debug: Check what data we have
print("=== DEBUGGING INFORMATION ===")
print(f"Total data shape: {data.shape}")
print(f"Date range in data: {data['date'].min()} to {data['date'].max()}")
print(f"Unique dates: {data['date'].nunique()}")
print(f"Unique stocks: {data['id'].nunique()}")

# Check if pred_out has any data
print(f"\nPrediction output shape: {pred_out.shape}")
print(f"Prediction output columns: {list(pred_out.columns) if not pred_out.empty else 'EMPTY'}")

if pred_out.empty:
    print("ERROR: No predictions were generated!")
    print("This could be due to:")
    print("1. Date filtering is too restrictive")
    print("2. No data in the specified date ranges")
    print("3. ARIMA model failed for all stocks")
else:
    print(f"Number of predictions: {len(pred_out)}")
    print(f"Sample predictions:\n{pred_out.head()}")
    
    # Check for missing values in predictions
    print(f"Missing values in ARIMA predictions: {pred_out['arima'].isna().sum()}")
    print(f"Missing values in actual returns: {pred_out[ret_var].isna().sum()}")
    
    # Only calculate metrics if we have data
    if len(pred_out) > 0:
        yreal = pred_out[ret_var].values
        ypred_arima = pred_out["arima"].values
        
        # Remove any NaN values
        mask = ~(np.isnan(yreal) | np.isnan(ypred_arima))
        yreal_clean = yreal[mask]
        ypred_clean = ypred_arima[mask]
        
        print(f"Clean data points: {len(yreal_clean)}")
        
        if len(yreal_clean) > 0:
            # Calculate R²
            r2_arima = 1 - np.sum(np.square((yreal_clean - ypred_clean))) / np.sum(np.square(yreal_clean))
            print(f"ARIMA OOS R²: {r2_arima:.6f}")
            
            # Calculate additional metrics
            mse_arima = mean_squared_error(yreal_clean, ypred_clean)
            rmse_arima = np.sqrt(mse_arima)
            mae_arima = np.mean(np.abs(yreal_clean - ypred_clean))
            
            print(f"ARIMA MSE: {mse_arima:.6f}")
            print(f"ARIMA RMSE: {rmse_arima:.6f}")
            print(f"ARIMA MAE: {mae_arima:.6f}")
        else:
            print("ERROR: No valid data points after removing NaN values!")
    else:
        print("ERROR: No predictions to evaluate!")

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(12, 8))

# Scatter plot
plt.subplot(2, 2, 1)
plt.scatter(yreal, ypred_arima, alpha=0.5)
plt.plot([yreal.min(), yreal.max()], [yreal.min(), yreal.max()], 'r--', lw=2)
plt.xlabel('Actual Returns')
plt.ylabel('Predicted Returns')
plt.title('ARIMA: Predicted vs Actual Returns')
plt.grid(True, alpha=0.3)

# Residuals plot
plt.subplot(2, 2, 2)
residuals = yreal - ypred_arima
plt.scatter(ypred_arima, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Returns')
plt.ylabel('Residuals')
plt.title('ARIMA: Residuals vs Predicted')
plt.grid(True, alpha=0.3)

# Time series plot (first 1000 observations)
plt.subplot(2, 2, 3)
n_plot = min(1000, len(yreal))
plt.plot(yreal[:n_plot], label='Actual', alpha=0.7)
plt.plot(ypred_arima[:n_plot], label='Predicted', alpha=0.7)
plt.xlabel('Observation')
plt.ylabel('Returns')
plt.title('ARIMA: Time Series (First 1000 obs)')
plt.legend()
plt.grid(True, alpha=0.3)

# Histogram of residuals
plt.subplot(2, 2, 4)
plt.hist(residuals, bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('ARIMA: Distribution of Residuals')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Print summary statistics
print("\n=== ARIMA Model Summary ===")
print(f"Total predictions: {len(pred_out)}")
print(f"R²: {r2_arima:.6f}")
print(f"RMSE: {rmse_arima:.6f}")
print(f"MAE: {mae_arima:.6f}")
print(f"\nActual returns - Mean: {np.mean(yreal):.6f}, Std: {np.std(yreal):.6f}")
print(f"Predicted returns - Mean: {np.mean(ypred_arima):.6f}, Std: {np.std(ypred_arima):.6f}")

# For timing purpose
print(f"\nEnd time: {datetime.datetime.now()}")
